In [12]:
import numpy as np
import matplotlib.pyplot as plt
import math
from scipy.stats import norm

# Case 3: Numéraires in a Binomial Tree & Monte Carlo

In [30]:
r = 0.01       
sigma = 0.15   
T = 10          
t = 0          
n = 100
guarantee = 100

In [31]:
def binomialTree_ORIGINAL(r, sigma, T, guarantee, n):

    dt = T / n
    R_dt = np.exp(r * dt)

    u = R_dt * np.exp(sigma * np.sqrt(dt))
    d = R_dt * np.exp(- sigma * np.sqrt(dt))
    
    p = (R_dt - d) / (u - d)
    q = 1 - p

    # Stock price matrix
    stockValues = 100 * u**np.arange(n + 1) * d**(n - np.arange(n + 1))

    # init option prices at maturity
    contractPrice = np.maximum(stockValues, guarantee)

    # calculate option prices at each node
    for i in np.arange(n,0,-1):
        contractPrice = (1/R_dt) * (p * contractPrice[1:i+1] + q * contractPrice[0:i])

    return contractPrice[0]

option_price = binomialTree_ORIGINAL(r, sigma, T, guarantee, n)
print("Option Price normal Binomial Model:", option_price)

Option Price normal Binomial Model: 113.45806426770127


## Numéraire =  Bank

In [32]:
# CASE 1: Numeraire Bank Account

def binomialTree_BankNum(r, sigma, T, guarantee, n):
    dt = T / n
    R_dt = np.exp(r * dt)

    u = R_dt * np.exp(sigma * np.sqrt(dt))
    d = R_dt * np.exp(-sigma * np.sqrt(dt))
    
    # Adjusted probability with B as numéraire
    pi_star = (R_dt - d) / (u - d)

    # money-market account values
    B = 100 * R_dt ** np.arange(n + 1)

    # Stock prices
    stockValues = 100 * u**np.arange(n + 1) * d**(n - np.arange(n + 1))

    # option prices 
    contractPrice = np.maximum(stockValues, guarantee)

    for i in np.arange(n, 0, -1):
        contractPrice = ((pi_star * contractPrice[1:i+1] + (1 - pi_star) * contractPrice[0:i] ) / B[i]) * B[i-1]

    return contractPrice[0]

In [33]:
option_price = binomialTree_BankNum(r, sigma, T, guarantee, n)
print("Option Price with Money-Market as Numéraire:", option_price)

Option Price with Money-Market as Numéraire: 113.45806426770169


# Numéraire: Stock

In [ ]:
def binomialTree_StockAsNumeraire(r, sigma, T, guarantee, n):
    dt = T / n
    R_dt = np.exp(r * dt)

    u = R_dt * np.exp(sigma * np.sqrt(dt))
    d = R_dt * np.exp(-sigma * np.sqrt(dt))

    pi_star = ((R_dt - d) * u) / ((u - d) * R_dt)

    # Stock price matrix
    stockValues = 100 * u**np.arange(n + 1) * d**(n - np.arange(n + 1))

    # option prices at maturity
    contractPrice = np.maximum(stockValues, guarantee)

    # option prices with adjusted probability
    for i in np.arange(n, 0, -1):
        contractPrice = (pi_star * contractPrice[1:i+1] / stockValues[1:i+1] + 
                         (1 - pi_star) * contractPrice[0:i] / stockValues[0:i]) * stockValues[1:i+1] / u

    return contractPrice[0]

In [29]:
binomialTree_StockAsNumeraire(r, sigma, T, guarantee, n)

np.float64(113.45806426770112)